## Import deps

In [ ]:
!pip install -U datasets mlable tokun llaminate

In [ ]:
!huggingface-cli login

%load_ext tensorboard

In [ ]:
import datetime
import functools
import itertools
import math
import os
import random
import urllib.request

import datasets as hd
import tensorflow as tf

import mlable.data
import mlable.metrics

import tokun.data
import tokun.evaluation
import tokun.meta
import tokun.model
import tokun.pipeline

import llaminate.model
import llaminate.pipeline
import llaminate.utils

In [ ]:
print("Tensorflow version " + tf.__version__)

## Setup the GPU / TPU

In [ ]:
# MIXED PRECISION #############################################################

tf.keras.mixed_precision.set_global_policy('mixed_bfloat16')

In [ ]:
# DEVICES #####################################################################

tf.debugging.set_log_device_placement(False)

CPU = tf.config.list_logical_devices('CPU')
GPU = tf.config.list_logical_devices('GPU')
TPU = tf.config.list_logical_devices('TPU')

if TPU:
    RESOLVER = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(RESOLVER)
    tf.tpu.experimental.initialize_tpu_system(RESOLVER)
    DISTRIBUTION_STRATEGY = tf.distribute.TPUStrategy(RESOLVER)
elif GPU:
    DISTRIBUTION_STRATEGY = tf.distribute.MirroredStrategy(GPU)
else:
    DISTRIBUTION_STRATEGY = tf.distribute.MirroredStrategy(CPU)

print(DISTRIBUTION_STRATEGY)

## Mode

In [ ]:
# TOGGLE ######################################################################

IMPORT = False
DOWNLOAD = False
TRAINING = True
BINARY = True

## Defining The Metadata

In [ ]:
# PIPELINE ####################################################################

BATCH_CONFIG = {
    'batch_size': 64,
    'drop_remainder': True,
    'num_parallel_calls': tf.data.AUTOTUNE,}

PIPELINE_CONFIG = {
    'batch_dim': BATCH_CONFIG['batch_size'],
    'sample_dim': 1024 * 8,
    'input_dim': 8,
    'data_weight': 1.0,
    'padding_weight': 0.0001,
    'separator': '\x1d',}

In [ ]:
# MODEL PARAMETERS ############################################################

LLAMINATE_CONFIG = {
  'num_layers': 12,
  'num_heads': 16,
  'input_dim': PIPELINE_CONFIG['input_dim'],
  'embed_dim': 1024,
  'head_dim': 1024 // 16,
  'hidden_dim': 1024 * 4,
  'epsilon': 1e-6,}

In [ ]:
# DERIVED PARAMETERS ##########################################################

LLAMINATE_META = {
    'version': '{}x{}x{}'.format(LLAMINATE_CONFIG['num_layers'], LLAMINATE_CONFIG['input_dim'], LLAMINATE_CONFIG['embed_dim']),
    'path': 'llaminate.keras',
    'url': '',}

In [ ]:
# TRAINING PARAMETERS #########################################################

OPTIMIZER_CONFIG = {
    'learning_rate': 0.001 * (0.1 if IMPORT else 1.0),
    'weight_decay': 0.1,
    'beta_1': 0.9,
    'beta_2': 0.95,
    'clipnorm': 1.0,}

SCHEDULER_CONFIG = {
    'initial_learning_rate': OPTIMIZER_CONFIG['learning_rate'],
    'decay_steps': 16384 * 3,
    'alpha': 0.1,
    'name': 'cosine_lr',
    'warmup_target': None,
    'warmup_steps': 0,}

METRICS_CONFIG = {
    'depth': 8,
    'threshold': 0.6,}

LOSS_CONFIG = {
    'from_logits': False,
    'label_smoothing': 0.,
    'axis': -1,
    'reduction': 'sum_over_batch_size',
    'name': 'loss',}

CHECKPOINT_CONFIG = {
    'filepath': LLAMINATE_META['path'],
    'monitor': 'val_loss',
    'mode': 'auto',
    'save_freq': 'epoch',
    'save_best_only': False,
    'save_weights_only': False,
    'verbose': 1,}

TENSORBOARD_CONFIG = {
    'log_dir': os.path.join('.logs/', *LLAMINATE_META['version'], datetime.datetime.now().strftime("%Y%m%d-%H%M%S")),
    'histogram_freq': 1,
    'embeddings_freq': 0,
    'profile_batch': (128, 256),
    'write_graph': False,
    'write_images': True,}

TRAINING_CONFIG = {
    'epochs': 4,
    'batch_size': None,
    'validation_split': None,
    'validation_freq': list(range(1, 9)),
    # 'class_weight': {__c: 1. if __c == 0 else 1. for __c in range(256)}, # there are 3 times more 0s than other bytes
    'verbose': 1,}

In [ ]:
# DATASETS ####################################################################

# TODO bigcode/the-stack
# TODO ArmelR/stack-exchange-instruction

DATASETS_CONFIG = {
    'pt-fineweb-edu': {
        'path': 'HuggingFaceFW/fineweb-edu',
        'name': 'sample-10BT',
        'splits': [f'train[{__p}%:{__p + 10}%]' for __p in range(0, 100, 10)],
        'features': ['text'],},
    # 'pt-wikipedia': {
    #     'path': 'wikimedia/wikipedia',
    #     'name': '20231101.en',
    #     'splits': [f'train[{__p}%:{__p + 9}%]' for __p in range(0, 80, 8)],
    #     'features': ['text'],},
    # 'tp-wikipedia-1': {
    #     'path': 'wikimedia/wikipedia',
    #     'name': '20231101.en',
    #     'splits': [f'train[{__p}%:{__p + 1}%]' for __p in range(80, 90, 1)],
    #     'features': ['text'],},
    # 'tp-wikipedia-2': {
    #     'path': 'wikimedia/wikipedia',
    #     'name': '20231101.en',
    #     'splits': [f'train[{__p}%:{__p + 1}%]' for __p in range(90, 100, 1)],
    #     'features': ['text'],},
    # 'ft-retro-ascii-art': {
    #     'path': 'jdpressman/retro-ascii-art-v1',
    #     'name': None,
    #     'train': 'train',
    #     'splits': [f'train[{__p}%:{__p + 10}%]+validation[{__p}%:{__p + 10}%]' for __p in range(0, 100, 10)],
    #     'features': ['prompt', 'art_aic'],},
    # 'ft-stack-exchange': {
    #     'path': 'Alignment-Lab-AI/Stack-Exchange-April',
    #     'name': None,
    #     'splits': [f'train[{__p}%:{__p + 10}%]' for __p in range(0, 100, 10)],
    #     'features': ['question', 'answer'],},
    'ft-math': {
        'path': 'hendrycks/competition_math',
        'name': None,
        'splits': [f'train[{__p}%:{__p + 10}%]+test[{__p}%:{__p + 10}%]' for __p in range(0, 100, 10)],
        'features': ['problem', 'solution'],},}

## Loading The Weights

In [ ]:
# DERIVED PARAMETERS ##########################################################

if IMPORT and DOWNLOAD:
    urllib.request.urlretrieve(LLAMINATE_META['url'], LLAMINATE_META['path'])

## Loading The Data

In [ ]:
# DOWNLOAD ####################################################################

DATASETS = {
    __name: [
        hd.load_dataset(path=__args['path'], name=__args['name'], split=__s).to_tf_dataset(shuffle=True, batch_size=None)
        for __s in __args['splits']]
    for __name, __args in DATASETS_CONFIG.items()}

## Checking The Data

In [ ]:
# STATS #######################################################################

STATS = {__n: mlable.data.stats(dataset=DATASETS[__n][0], features=DATASETS_CONFIG[__n]['features'], count=2048) for __n in DATASETS}

print(STATS)

In [ ]:
__b = iter(DATASETS['pt-fineweb-edu'][0])
next(__b)

## Preprocess

In [ ]:
# ITERATE #####################################################################

with DISTRIBUTION_STRATEGY.scope():
    for __name in DATASETS:
        # specialized preprocessing fn
        __preprocess = llaminate.pipeline.preprocess_factory(
            features=DATASETS_CONFIG[__name]['features'],
            **PIPELINE_CONFIG)
        # apply
        for __idx in range(len(DATASETS[__name])):
            DATASETS[__name][__idx] = DATASETS[__name][__idx].batch(**BATCH_CONFIG).map(__preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# CONCATENATE #################################################################

FINE_TRAIN = functools.reduce(lambda __l, __r: __l.concatenate(__r), DATASETS['pt-fineweb-edu'][:-1])
FINE_TEST = DATASETS['pt-fineweb-edu'][-1]

DATASET_TRAIN = functools.reduce(lambda __l, __r: __l.concatenate(__r), [DATASETS[__n][__i] for __n in (set(DATASETS.keys()) - {'ft-retro-ascii-art'}) for __i in range(len(DATASETS[__n]) - 1)]) # - {'pt-wikipedia'}
DATASET_TEST = functools.reduce(lambda __l, __r: __l.concatenate(__r), [DATASETS[__n][-1] for __n in (set(DATASETS.keys()) - {'ft-retro-ascii-art'})]) # - {'pt-wikipedia'}

In [ ]:
# CHECK DATASET ###############################################################

__X, __T, __W = next(iter(FINE_TRAIN.take(1)))

print(FINE_TRAIN.element_spec)
print(FINE_TEST.element_spec)

print(DATASET_TRAIN.element_spec)
print(DATASET_TEST.element_spec)

print('fine: {:,} / {:,} samples'.format(FINE_TRAIN.cardinality().numpy(), FINE_TEST.cardinality().numpy()))
print('total: {:,} / {:,} samples'.format(DATASET_TRAIN.cardinality().numpy(), DATASET_TEST.cardinality().numpy()))

## Initializing The Model

In [ ]:
# METRICS #####################################################################

_Accuracy = mlable.metrics.BinaryGroupAccuracy if BINARY else mlable.metrics.RawGroupAccuracy
_Loss = tf.keras.losses.BinaryCrossentropy if BINARY else tf.keras.losses.MeanSquaredError

In [ ]:
# OVERALL SCOPE ###############################################################

with DISTRIBUTION_STRATEGY.scope():
    # COSINE LR ###############################################################
    cosine_lr = tf.keras.optimizers.schedules.CosineDecay(**SCHEDULER_CONFIG)
    OPTIMIZER_CONFIG['learning_rate'] = cosine_lr

    # METRICS #################################################################
    byte_accuracy = _Accuracy(group=1, name='byte_accuracy', **METRICS_CONFIG)
    character_accuracy = _Accuracy(group=4, name='character_accuracy', **METRICS_CONFIG)
    token_accuracy = _Accuracy(group=PIPELINE_CONFIG['input_dim'], name='token_accuracy', **METRICS_CONFIG)

    # WEIGHTS #################################################################
    LLAMINATE = llaminate.model.Transformer(**LLAMINATE_CONFIG)
    if IMPORT and os.path.isfile(LLAMINATE_META['path']): LLAMINATE = tf.keras.models.load_model(LLAMINATE_META['path'], compile=False)

    # BUILD ###################################################################
    LLAMINATE(__X)

    # COMPILE #################################################################
    LLAMINATE.compile(
        optimizer=tf.keras.optimizers.AdamW(**OPTIMIZER_CONFIG),
        loss=_Loss(**LOSS_CONFIG),
        weighted_metrics=[byte_accuracy, character_accuracy, token_accuracy])

In [ ]:
LLAMINATE.summary()

## Train

In [ ]:
# TRAIN #######################################################################

if TRAINING:
    with DISTRIBUTION_STRATEGY.scope():
        # callbacks
        cp_callback = tf.keras.callbacks.ModelCheckpoint(**CHECKPOINT_CONFIG)
        tb_callback = tf.keras.callbacks.TensorBoard(**TENSORBOARD_CONFIG)
        # model fitting
        TRAINING_HISTORY = LLAMINATE.fit(
            x=DATASETS['pt-fineweb-edu'][0].prefetch(tf.data.AUTOTUNE),
            validation_data=DATASETS['pt-fineweb-edu'][-1].take(128).prefetch(tf.data.AUTOTUNE),
            callbacks=[cp_callback, tb_callback],
            **TRAINING_CONFIG)

## Dataviz

In [ ]:
__i = iter(DATASETS['pt-fineweb-edu'][-1])

In [ ]:
__x, __t, __w = next(__i)
__y = LLAMINATE(__x)

In [ ]:
__s = llaminate.pipeline.postprocess(__y)
__s[:4]

In [ ]:
__s = llaminate.pipeline.postprocess(__t)
__s[:4]

In [ ]:
# DATA ########################################################################

SAMPLES = [
    """위키백과, 우리 모두의 백과사전.\nt-분포 확률적 임베딩(t-SNE)은 데이터의 차원 축소에 사용되는 기계 학습 알고리즘 중 하나로, 2002년 샘 로이스Sam Rowise와 제프리 힌튼에 의해 개발되었다.[1] t-SNE는 비선형 차원 축소 기법으로, 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용된다. 구체적으로 t-SNE는 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑한다.""",
    """class Encoder(tf.keras.models.Model):\n    def __init__(self, depth: int, token_dim: int, encoding_dim: int, embedding_dim: int, latent_dim: int, batch_dim: int=None, attention: bool=False, **kwargs) -> None:\n        super(Encoder, self).__init__(**kwargs)\n        self._encoder = tf.keras.Sequential([\n            tf.keras.Input(shape=(encoding_dim,), batch_size=batch_dim, name='input'), # (B * G ^ D, U)\n            tf.keras.layers.Dense(units=embedding_dim, activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer=None, name='embed-1'),] # (B * G ^ D, U) => (B * G ^ D, E)\n            + [tokun.layers.TokenizeBlock(left_axis=-2, right_axis=-1, token_dim=token_dim, latent_dim=latent_dim, attention=attention, name='tokenize' + (__i + 1) * '-4') for __i in range(depth)]) # (B * G ^ i, E) => (B * G ^ (i-1), E)\n\n    def call(self, x: tf.Tensor) -> tf.Tensor:\n        return self._encoder(x)\n""",
    """class AutoEncoder(tf.keras.models.Model):\n    def __init__(self, token_dim: int, encoding_dim: int, embedding_dim: int, latent_dim: int, batch_dim: int=None, **kwargs) -> None:\n        super(AutoEncoder, self).__init__(**kwargs)\n        self._encoder = Encoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n        self._decoder = Decoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n\n    def call(self, x: tf.Tensor) -> tf.Tensor:\n        return self._decoder(self._encoder(x))""",
    """class AutoEncoder(tf.keras.models.Model):\n  def __init__(self, token_dim: int, encoding_dim: int, embedding_dim: int, latent_dim: int, batch_dim: int=None, **kwargs) -> None:\n    super(AutoEncoder, self).__init__(**kwargs)\n    self._encoder = Encoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n    self._decoder = Decoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n\n  def call(self, x: tf.Tensor) -> tf.Tensor:\n    return self._decoder(self._encoder(x))"""]

In [ ]:
# CACHE #######################################################################

__cache = llaminate.utils.create_cache(batch_dim=N_BATCH_DIM, cache_dim=N_CACHE_DIM, head_dim=N_HEAD_DIM, num_layers=N_LAYERS_NUM, num_heads=N_HEADS_NUM)
__step = 4

In [ ]:
# PREPROCESS ##################################################################

__prompt = """Skynet is an artificial neural network-based conscious group mind and artificial general superintelligence system that serves as the antagonistic force of the Terminator franchise."""
__inputs = tokun.pipeline.preprocess(text=__prompt, token_size=PIPELINE_CONFIG['sample_dim'], expand_dims=[1], output_dtype=tf.uint8)

In [ ]:
__inputs = mlable.shaping.divide(__inputs, input_axis=-2, output_axis=-1, factor=PIPELINE_CONFIG['input_dim'], insert=True)

In [ ]:
# PREDICT #####################################################################

__predictions = LLAMINATE(__inputs)
__outputs = llaminate.pipeline.postprocess(__predictions)

In [ ]:
tokun.pipeline.chunk(__prompt, size=4)

In [ ]:
__outputs

In [ ]:
__batch = iter(DATASETS['pt-wikipedia'][1])

In [ ]:
__x, __y, __m = next(__batch)
__p = LLAMINATE(inputs=__x, training=True, mask=None)

In [ ]:
__yt = tokun.pipeline.interpret(__y, binary=True)
__yp = tokun.pipeline.interpret(__p, binary=True)
__it = tokun.pipeline.decode(__x)
__ot = tokun.pipeline.decode(__yt)
__op = tokun.pipeline.decode(__yp)

In [ ]:
print(__it[:2])
print(__ot[:2])
print(__op[:2])

## Logs

In [ ]:
%tensorboard --logdir .logs